## Install packages

In [1]:
# !pip install transformers

In [2]:
%load_ext autoreload
%autoreload 2

import os
import re

# from transformers import get_linear_schedule_with_warmup
# from transformers.optimization import AdamW
import sys
from datetime import datetime
from pathlib import Path
from sys import platform

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import src.utils as utils
import torch
from torch.utils.data import DataLoader, Dataset

%matplotlib inline

## Set up GPU

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"There are {torch.cuda.device_count()} GPU(s) available.")
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce GTX 1070


## Load Dataset

In [4]:
data_path = Path("data")

all_train_df = pd.read_csv(data_path / "train.csv")
all_test_df = pd.read_csv(data_path / "test.csv")
all_val_df = pd.read_csv(data_path / "val.csv")

print(f"train: {len(all_train_df)}")
print(f"test: {len(all_test_df)}")
print(f"val: {len(all_val_df)}")

train: 67349
test: 1821
val: 872


In [5]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(all_train_df, test_size=0.1, random_state=23)
test_df = all_val_df

# Reset all indices
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f"train: {len(train_df)}")
print(f"val: {len(val_df)}")
print(f"test: {len(test_df)}")

train: 60614
val: 6735
test: 872


In [6]:
train_df.sentence.apply(lambda d: len(d.split())).max()

52

In [7]:
val_df.sentence.apply(lambda d: len(d.split())).max()

51

In [8]:
test_df.sentence.apply(lambda d: len(d.split())).max()

47

# Fine-Tuned Bert Classifier

## PyTorch DataLoader

In [9]:
%%time
from src.datasets import SST2Dataset
from torch.utils.data import (
    DataLoader,
    RandomSampler,
    SequentialSampler,
    TensorDataset,
    random_split,
)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32
MAX_LEN = 64
FRAC = 1

device = utils.get_device()

# Train data
print("Train")
train_data = SST2Dataset.create_dataset(
    "train", device, train_df, max_seq_len=MAX_LEN, frac=FRAC
)
train_dataloader = DataLoader(
    train_data, sampler=RandomSampler(train_data), batch_size=batch_size
)

# Val data
print("Validation")
val_data = SST2Dataset.create_dataset(
    "val", device, val_df, max_seq_len=MAX_LEN, frac=FRAC
)
val_dataloader = DataLoader(val_data, sampler=SequentialSampler(val_data))

# Test data
print("Test")
test_data = SST2Dataset.create_dataset("test", device, test_df, max_seq_len=MAX_LEN)
test_dataloader = DataLoader(test_data, sampler=SequentialSampler(test_data))

Train


100%|████████████████████████████████████████████████████████████████████████████████████████████| 60614/60614 [00:04<00:00, 13847.50it/s]


Validation


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 6735/6735 [00:00<00:00, 12217.37it/s]


Test


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 872/872 [00:00<00:00, 10408.65it/s]

CPU times: user 6.29 s, sys: 490 ms, total: 6.78 s
Wall time: 7.05 s


## Train

In [10]:
from src.BertClassifier import BertClassifier
from transformers import AdamW, get_linear_schedule_with_warmup


def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler."""
    # Instantiate Bert Classifier
    device = utils.get_device()
    bert_classifier = BertClassifier(freeze_bert=True)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(
        bert_classifier.classifier.parameters(),
        lr=5e-5,  # Default learning rate
        eps=1e-8,  # Default epsilon value
    )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=total_steps  # Default value
    )
    return bert_classifier, optimizer, scheduler

def load_model():
    device = utils.get_device()
    classifier = torch.load("classifier")
    # Create the optimizer
    optimizer = AdamW(
        bert_classifier.classifier.parameters(),
        lr=5e-5,  # Default learning rate
        eps=1e-8,  # Default epsilon value
    )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # TODO use a different type of scheduler when retraining the model
    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=total_steps  # Default value
    )
    
    return classifier, optimizerr, scheduler
    

In [ ]:
%%time
epochs = 10

utils.set_seed(42)  # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=epochs)
history = utils.train(
    bert_classifier,
    optimizer,
    scheduler,
    train_dataloader,
    val_dataloader,
    epochs=epochs,
    evaluation=True,
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/pcao/miniconda3/envs/pytorch/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_dep

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1895/1895 [02:14<00:00, 14.10batch/s]


   1    |  1894   |   0.526085   |     -      |     -     |  134.42  
----------------------------------------------------------------------
   1    |    -    |   0.526085   |  0.407122  |   83.30   |  176.31  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1895/1895 [02:04<00:00, 15.18batch/s]


   2    |  1894   |   0.405027   |     -      |     -     |  124.87  
----------------------------------------------------------------------
   2    |    -    |   0.405027   |  0.367280  |   84.20   |  165.47  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1895/1895 [02:05<00:00, 15.15batch/s]


   3    |  1894   |   0.384332   |     -      |     -     |  125.12  
----------------------------------------------------------------------
   3    |    -    |   0.384332   |  0.355725  |   84.90   |  165.12  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------


 77%|████████████████████████████████████████████████████████████████████████▌                     | 1462/1895 [01:35<00:28, 15.34batch/s]

In [ ]:
epoc_length = len(history["train_loss"])
x = np.arange(1, epoc_length + 1)


plt.plot(x, history["train_loss"])
plt.plot(x, history["val_loss"])
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
plt.plot(np.arange(1, len(train_loss) + 1), train_loss)

In [ ]:
plt.plot(np.arange(1, len(val_loss) + 1), val_loss)

In [ ]:
# Save model parameters

model_params_path = Path("model_params")

now = datetime.now().strftime("%m-%d-%Y_%H%M%S")

# torch.save(bert_classifier.classifier.state_dict(), model_params_path / f"{now}-classifier")

## Evaluate

In [ ]:
from src.BertClassifier import bert_predict
from src.utils import evaluate_roc

# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader, device)

# Evaluate the Bert classifier
# evaluate_roc(probs, y_val[: len(probs)])
evaluate_roc(probs, test_data.labels.cpu())

In [ ]:
# Compute predicted probabilities on the validation set
probs = bert_predict(bert_classifier, val_dataloader, device)

# Evaluate the Bert classifier
evaluate_roc(probs, val_data.labels.cpu())

In [ ]:
len(val_data)

In [ ]:
len(test_data)

# LOO

In [ ]:
# guids = val_data.guids
# labels = val_data.labels

In [ ]:
%%time
loo = val_data.leave_one_out(3018)

In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader, device)

In [ ]:
probs